# Understanding Object Oriented Languages by Building One
Many misconceptions about object-oriented programming exist, which often lead to sub-optimal code in expressiveness and maintanability. Unfortunately, the source of all Truth (Wikipedia) has only a very general description of the pattern, and does not help understand the core value of this paradigm.

This tutorial aims at showing the core concepts of OO, and how it can be leveraged to build extensible and powerful systems.

After identifying a few common concepts (incorrectly) associated with object oriented programs, we will build a fully functional object-oriented language, with classes, inheritance and multiple dispatch.

In [1]:
# Syntactic sugar: access elements of the dictionary with the same syntax than attributes
class MagicDict:
    def __init__(self, d):
        self._d = d
    def __getattr__(self, attr):
        if attr == "_d":
            super().__getattr__(attr)
            return
        return self._d[attr]
    def __setattr__(self, attr, value):
        if attr == "_d":
            super().__setattr__(attr, value)
            return
        if attr not in self._d:
            raise Exception("No such key " + attr)
        self._d[attr] = value
    def __str__(self):
        return str(self._d)

## Misconception 1: Classes as Namespaces


A sure way to spot misunderstanding about object orientation is to look at a logging class: the class will be full of static methods! What the programmer is trying to achieve is prefixing functions calls (i.e. `log.error`), which does not need object orientation features.

Namespaces are much better provided as modules or packages (if the language supports them), or with a simple prefix (for example C).

In [2]:
class Log:
    "A class misused as namespace"
    @staticmethod
    def info(message):
        print("INFO", message)
    @staticmethod
    def error(message):
        print("ERROR", message)

def log_info(message):
    print("INFO", message)
def log_error(message):
    print("ERROR", message)
    
Log.info("a class is not a namespace!")
log_info("a prefix would do just as well…")

INFO a class is not a namespace!
INFO a prefix would do just as well…


A variant of this pattern is a class with only values — or even worse, with plainly redundant getters and setters.

In [3]:
class Person:
    "A bag of values"
    def __init__(self, name, age):
        self._name = name
        self._age = age
    def setName(self, name):
        self._name = name
    def getName(self):
        return self._name
    def setAge(self, age):
        self._age = age
    def getAge(self):
        return self._age
    def __str__(self):
        return str({"name": self._name, "age": self._age})
    
John = Person("John Doe", 21)
John.setAge(23)
print(John)

Jack = MagicDict({"name": "Jack the Ripper", "age": 21})
Jack.age = 23
print(Jack)

{'name': 'John Doe', 'age': 23}
{'name': 'Jack the Ripper', 'age': 23}


## Misconception 2: Object-Orientation Deals with Information Hiding (aka Encapsulation)

Information hiding has multiple meanings, and is often associated with "Enterprise" programming, where team jealously keep tight control over their code and limit interactions with other pieces of code. Lets put that thought aside by reminding that, as long as the memory is shared, there is simply no boundary that can be enforced — therefore, those `private` keywords are basically just documentation.

Hiding the details of the implementation, and therefore allowing users of the library to work at a higher level of abstraction is indeed a useful feature, and deserves more attention. Let us look at the (archetypical) example of the counter: a value that can only be modified through an `inc` method, and inspected through a `count` one. This example can completely be implemented by a simple closure over the variable.

In [28]:
def Counter():
    "A closure encapsulate our state"
    count = 0
    def add():
        nonlocal count
        count += 1
    def pcount():
        return count
    return MagicDict({
        "add": add,
        "count": pcount,
    })

c1 = Counter()
c1.add()
c1.count()

1

This example is showing an interesting promise though: what if we want to start using a gauge (i.e. a counter that can go both up and down)? It looks like we can continue apply the same technique.


In [5]:
def Gauge():
    count = 0
    def add():
        nonlocal count
        count += 1
    def decr():
        nonlocal count
        if count > 0:
            count -= 1
    def pcount():
        return count
    return MagicDict({
        "add": add,
        "count": pcount,
        "decr": decr,
    })

def MyServer(counter):
    "starts a new server, and count the requests"
    counter.add()
    
c = Counter()
MyServer(c)
print("Counter", c.count())

g = Gauge()
MyServer(g)
g.decr()
print("Gauge", g.count())

Counter 1
Gauge 0


What is nice here, is the the call to `add` is _independent_ of the underlying closure / implementation behind it; and that the `MyServer` definition accepts *both* a gauge and a counter — this is called dynamic dispatch, and is a very powerful implementation of the critical concept of abstraction of implementation (cf B.Liskov).

## Lets Start Baking
Now that we have understood that the single most interesting feature of object orientation is the dynamic dispatch, which allow to execute a different piece of code each time it is called.

Now comes the real fun: building our own dispatch system, and therefore our object-oriented mini language. Say hello to Cookie!

In [6]:
class Cookie:  # class is used to pretty-print only!
    def __str__(self):
        return self._name
    
    def __init__(self, name, operations):
        self._name = name
        self._operations = operations
        
    def do(self, op, *args):
        if not op in self._operations:
            raise Exception("no such operation", op)
        return self._operations[op](*args)

def bake_butter():
    return "cooking at 100°"

def bake_chocolate(color):
    if color == "white":
        return "cooking at 90°"
    else:
        return "cooking at 80°"

c1 = Cookie("ButterCookie", {"bake": bake_butter})
c2 = Cookie("ChocolateCookie", {"bake": bake_chocolate})
print(str(c1) + ":" + c1.do("bake"), str(c2) + ":" + c2.do("bake", "white"))

ButterCookie:cooking at 100° ChocolateCookie:cooking at 90°


But, wait! This is cheating: we currently have only methods that act on their parameters (static methods). This is loosing all the encapsulation power we have in the closure!

Lets add values to the cookie, and make an automated setter and getter since we are at it.

In [42]:
class Cookie2:
    def __str__(self):
        return self._name
    
    def __init__(self, name, operations, values):
        self._name = name
        self._values = values
        self._operations = operations
        
    def do(self, op, *args):
        if op.startswith("set"):
            self._values[op[3:].lower()] = args[0]
        elif op.startswith("get"):
            return self._values[op[3:].lower()]
        else:
            if not op in self._operations:
                raise Exception("no such operation", op)
            return self._operations[op](self, *args)
        
def bake_nuts(self):
    return "baking " + str(self.do("getWeight") * 12) + " minutes"

nCookie = Cookie2("Nuts cookie", {"bake": bake_nuts}, {"weight": 0.2})
print(nCookie.do("bake"))
nCookie.do("setWeight", 0.3)
print(nCookie.do("bake"))

baking 2.4000000000000004 minutes
baking 3.5999999999999996 minutes


Cooking one cookie at a time is OK, but a bit … limited, right? (OK, Javascript stops here, but why should we?) Let's make many cookies at a time, using the tool for the job.

In [24]:
class cookie:
    def __str__(self):
        return self._cutter.printCookie()

    def __init__(self, cutter, attr):
        self._cutter = cutter
        self._values = attr
        
    def do(self, op):
        if op.startswith("set"):
            self._values[op[3:].lower()] = args[0]
        elif op.startswith("get"):
            return self._values[op[3:].lower()]
        else:
            return self._cutter.do(self, op)

In [26]:
class CookieCutter:
    def __str__(self):
        return self._name + " cutter"
    
    def __init__(self, name, operations):
        self._name = name
        self._operations = operations
        
    def do(self, cookie, op, *args):
        if not op in self._operations:
            raise Exception("no such operation", op)
        return self._operations[op](cookie, *args)
    
    def printCookie(self):
        return "I am a " + self._name + " cookie"
    
    def newCookie(self, attr):
        c = cookie(self, attr)
        return c

def bake_macadamia(cookie):
    if cookie.do("getColor") == "white":
        return str(30)
    else:
        return str(100)

macadamiaCookieCutter = CookieCutter("MacadamiaCookie", {"bake": bake_macadamia})
noonCookie = macadamiaCookieCutter.newCookie({"color": "white"})
midnightCookie = macadamiaCookieCutter.newCookie({"color": "dark"})

print(str(noonCookie) , str(midnightCookie))
print(noonCookie.do("bake"), midnightCookie.do("bake"))

I am a MacadamiaCookie cookie I am a MacadamiaCookie cookie
white
dark
30 100


- inheritance
- parameters dispatch